# AI Guard Bot - Jarvis Security System

**Course:** EE782 - Advanced Machine Learning  
**Project:** AI Room Guard with Multi-Modal Intelligence  
**Team:** Aatmaj  
**Date:** November 2025  
**Hardware:** NVIDIA GeForce RTX 4050 Laptop GPU (6.4GB VRAM)

---

## Executive Summary

This project implements an intelligent security system that combines voice activation, face recognition, and conversational AI to monitor and respond to intruders. The system, named "Jarvis," uses three state-of-the-art models optimized for GPU execution:

- **Whisper Medium** (1.5GB VRAM) - Voice command recognition
- **FaceNet InceptionResnetV1** (0.2GB VRAM) - Real-time face recognition at 18-20 FPS
- **Llama3.1:8b** (4.7GB VRAM) - Context-aware conversational responses

The system achieves **100% GPU utilization** with optimal memory allocation and implements a three-level escalation protocol for handling unknown individuals.

## System Architecture

### State Flow Diagram

```
┌──────────────────────────────────────────────────────────────┐
│                         IDLE STATE                            │
│                  (Model Loading & Setup)                      │
└────────────────────────┬─────────────────────────────────────┘
                         │
                         ▼
┌──────────────────────────────────────────────────────────────┐
│                    M1: VOICE ACTIVATION                       │
│  • Whisper transcribes audio (3s chunks)                     │
│  • Similarity matching with activation phrases               │
│  • Threshold: 0.6 for activation                             │
│  • Anti-hallucination: logprob + condition_on_previous       │
└────────────────────────┬─────────────────────────────────────┘
                         │ "guard mode" detected
                         ▼
┌──────────────────────────────────────────────────────────────┐
│                 M2: FACE RECOGNITION                          │
│  • FaceNet processes faces (every 2 frames)                  │
│  • Cosine similarity matching (threshold: 0.40)              │
│  • Green box: Trusted (confidence ≥ 0.75)                    │
│  • Red box: Unknown                                           │
│  • Monitors for deactivation command + trusted face          │
│  • Accumulated timer: Unknown present >2s → M3               │
└────────────────────────┬─────────────────────────────────────┘
                         │ Unknown >2s
                         ▼
┌──────────────────────────────────────────────────────────────┐
│                M3: LLM ESCALATION PROTOCOL                    │
│  Level 1 (2s):  Initial contact + information gathering      │
│  Level 2 (4s):  Context-aware warning + exit demand          │
│  Level 3 (6s):  Final warning + authority notification       │
│  • Jarvis personality system                                 │
│  • Context tracking (name, purpose extraction)               │
│  • 20-word response limit                                     │
│  • 30s cooldown after completion                              │
└────────────────────────┬─────────────────────────────────────┘
                         │ Trusted face OR deactivation
                         ▼
                    Return to IDLE/M1
```

### Thread Architecture

- **Main Thread:** State machine controller
- **Audio Thread:** Continuous 3s chunk transcription (background)
- **Camera Thread:** 20 FPS video capture with OpenCV display
- **Face Thread:** Async face detection/encoding (every 2 frames)

**State Machine Implementation**
The state transitions shown above are controlled by the GuardBotStateMachine class. This class manages the entire lifecycle using handler functions for each state and accumulated timers for escalation.

**Key Components:**

**State Enum:** Defines IDLE, M1_VOICE, M2_FACE, M3_ESCALATION states

**AccumulatedTimer:** Tracks unknown face presence time (pauses when face disappears, resets after 7s absence)

**State Handlers:** `handle_idle()`, `handle_m1()`, `handle_m2()`, `handle_m3()`

**Timing Constants:** Level 1 (2s), Level 2 (4s), Level 3 (6s), Cooldown (30s)

This implementation ensures smooth transitions between states while maintaining precise timing control for escalation protocols.

In [ ]:
# Key Excerpt: State Machine and Timer Implementation

from enum import Enum
import threading
import time

class State(Enum):
    """Define all possible system states"""
    IDLE = "idle"
    M1_VOICE = "m1_voice"
    M2_FACE = "m2_face"
    M3_ESCALATION = "m3_escalation"

class AccumulatedTimer:
    """Timer with pause/resume/reset logic for face presence tracking"""
    def __init__(self, target_seconds):
        self.target = target_seconds
        self.accumulated = 0.0
        self.last_seen_time = None
        self.is_active = False
        self.lock = threading.Lock()
    
    def update(self, face_present):
        """Update timer based on face presence"""
        with self.lock:
            current_time = time.time()
            
            if face_present:
                if not self.is_active:
                    # Face just appeared, start timer
                    self.is_active = True
                    self.last_seen_time = current_time
                else:
                    # Accumulate time while face is continuously present
                    delta = current_time - self.last_seen_time
                    self.accumulated += delta
                    self.last_seen_time = current_time
            else:
                if self.is_active:
                    # Face disappeared, check how long it's been gone
                    time_gone = current_time - self.last_seen_time
                    if time_gone > 7.0:  # 7s absence threshold
                        print("⏸️  Face absent >7s, resetting timer")
                        self.reset()
            
            # Return True if target time reached
            return self.accumulated >= self.target
    
    def reset(self):
        """Reset timer to initial state"""
        self.accumulated = 0.0
        self.is_active = False
        self.last_seen_time = None

class GuardBotStateMachine:
    """Main state machine controller"""
    def __init__(self):
        self.state = State.IDLE
        self.unknown_timer = AccumulatedTimer(2.0)  # 2s for Level 1
        self.cooldown_active = False
        self.cooldown_start = 0
        self.shutdown_requested = False
    
    def run(self):
        """Main event loop - 50ms cycle time"""
        while not self.shutdown_requested:
            if self.state == State.IDLE:
                self.handle_idle()  # Personality selection + model loading
            
            elif self.state == State.M1_VOICE:
                self.handle_m1()  # Audio thread listens for activation
            
            elif self.state == State.M2_FACE:
                self.handle_m2()  # Face monitoring + deactivation detection
            
            elif self.state == State.M3_ESCALATION:
                self.handle_m3()  # 3-level escalation protocol
            
            time.sleep(0.05)  # 50ms loop prevents CPU thrashing

# Escalation timing constants (in seconds)
ESCALATION_TIMING = {
    "level1_presence": 2,   # Trigger L1 after 2s unknown presence
    "level2_presence": 4,   # Trigger L2 after 4s unknown presence
    "level3_presence": 6,   # Trigger L3 after 6s unknown presence
    "face_loss_reset": 7,   # Reset timer if face absent >7s
    "cooldown_duration": 30 # 30s cooldown after L3 completion
}

## Implementation Walkthrough

### 1. Model Initialization & GPU Optimization

The system uses a specific loading sequence to avoid CUDA out-of-memory errors and ensure optimal VRAM allocation.

**Loading Strategy:** Ollama → FaceNet → Whisper (GPU)

**Why this order?**
1. **Ollama first** - Pre-loads the largest model (4.7GB) before PyTorch claims GPU memory
2. **FaceNet second** - Lightweight (200MB), loaded with `torch.backends.cudnn.benchmark = True`
3. **Whisper last** - Loaded after Ollama to avoid CUDA conflicts (1.5GB)

In [ ]:
# Key Excerpt: Model Loading Sequence from main.py

def __init__(self):
    # GPU device initialization
    self.device = torch.device("cuda:0")

    # [1/3] Load Ollama FIRST - prevents CUDA OOM
    print("📥 [1/3] Connecting to Ollama (Llama3.1:8b) - 4.7GB...")
    ollama.generate(model='llama3.1:8b', prompt='Initialize', stream=False)

    # [2/3] Load FaceNet - Enable cuDNN optimization
    print("📥 [2/3] Loading FaceNet InceptionResnetV1 (GPU) - 200MB...")
    self.face_encoder = InceptionResnetV1(pretrained='vggface2').to(self.device).eval()
    torch.backends.cudnn.benchmark = True  # 🚀 30% speedup

    # [3/3] Load Whisper LAST - Avoids PyTorch/CUDA conflicts  
    print("📥 [3/3] Loading Whisper Medium (GPU) - 1.5GB...")
    self.whisper_model = whisper.load_model("medium", device=self.device)

    # Final VRAM allocation: 6.4GB / 6.4GB (100% optimal)

### Screenshot: Model Loading

![Model Loading](screenshots/loading_screen.png)

**Key observations:**
- Sequential loading prevents CUDA memory fragmentation
- Total VRAM: 6.4GB (100% optimal utilization)
- Ollama: ~4.7GB | FaceNet: ~0.2GB | Whisper: ~1.5GB
- Loading time: ~39.8s total (Ollama 39s + FaceNet 0.9s + Whisper 27s)
- Strategy avoids "CUDA out of memory" errors

### 2. Voice Activation (M1) - Whisper Anti-Hallucination

The audio thread continuously transcribes 3-second audio chunks using Whisper. A critical challenge was preventing **hallucinations** (Whisper inventing phrases like "Thanks for watching!" when there's silence or ambiguous audio).

**Solution:** Stricter Whisper parameters

In [ ]:
# Key Excerpt: Anti-Hallucination Whisper Configuration

def transcribe_audio_chunk(self, duration=3):
    '''Whisper transcription with hallucination prevention'''
    # Capture 3s of audio
    frames = []
    chunks_needed = int(SAMPLE_RATE / CHUNK_SIZE * duration)
    for _ in range(chunks_needed):
        data = self.stream.read(CHUNK_SIZE, exception_on_overflow=False)
        frames.append(np.frombuffer(data, dtype=np.int16))

    audio_np = np.concatenate(frames).astype(np.float32) / 32768.0

    # 🔧 Critical parameters to prevent hallucination
    result = self.whisper_model.transcribe(
        audio_np,
        language="en",
        fp16=True,
        no_speech_threshold=0.8,             # Stricter silence detection
        logprob_threshold=-0.5,              # Require 60% confidence
        condition_on_previous_text=False,    # No context bias
        temperature=0.0,                     # Deterministic output
        compression_ratio_threshold=2.4      # Reject repetitive text
    )

    transcription = result["text"].strip()

    # Additional filter: Minimum 2 words
    words = transcription.split()
    if len(words) < 2:
        return ""  # Reject single-word hallucinations

    return transcription

### Activation Command Detection

Once Whisper transcribes audio, the system uses **sequence-based similarity matching** to detect activation phrases.

**Activation phrases:**
- "guard mode"
- "activate guard"
- "start guard"
- "guard my room"
- "activate security"
- "start guarding"

**Similarity algorithm:**
- 70% SequenceMatcher ratio (substring matching)
- 30% word intersection (keyword matching)
- Combined threshold: 0.6

In [ ]:
# Key Excerpt: Similarity-Based Command Detection

def calculate_similarity(text, phrases):
    '''Fuzzy matching for voice commands'''
    text = text.lower().strip()
    best_similarity = 0.0
    best_phrase = ""

    for phrase in phrases:
        # Sequence matching (handles typos/variations)
        seq_similarity = SequenceMatcher(None, text, phrase).ratio()

        # Word intersection (keyword matching)
        text_words = set(text.split())
        phrase_words = set(phrase.split())
        word_similarity = len(text_words.intersection(phrase_words)) / max(len(phrase_words), 1)

        # Weighted combination (favor sequence matching)
        combined_score = 0.7 * seq_similarity + 0.3 * word_similarity

        if combined_score > best_similarity:
            best_similarity = combined_score
            best_phrase = phrase

    return best_similarity, best_phrase

# Process transcription
similarity, matched_phrase = calculate_similarity(transcription, ACTIVATION_PHRASES)
if similarity >= 0.6:
    activate_guard()  # Transition to M2

### Screenshot: Successful Activation

![Activation](screenshots/activation.png)

**Key observations:**
- Transcription: "Activate Guard Mode"
- Similarity score: 0.89 (matched: "activate guard")
- Guard mode activated successfully
- Camera and face threads started (20 FPS)
- No false activations due to hallucination prevention

### 3. Face Recognition (M2) - Real-Time Processing

The system uses **FaceNet InceptionResnetV1** for face recognition with a multi-threaded architecture to achieve 18-20 FPS.

**Architecture:**
- **CameraThread:** Captures frames at 20 FPS, draws overlays, displays OpenCV window
- **FaceProcessingThread:** Processes every 2nd frame (10 Hz) for face detection/encoding
- Shared memory with thread-safe locks

**Recognition Pipeline:**
1. Haar Cascade detects face locations
2. FaceNet encodes face to 512-dim vector
3. Cosine similarity matching with known embeddings
4. Confidence threshold: 0.75 for "trusted" classification

In [ ]:
# Key Excerpt: Face Recognition Pipeline

class FaceProcessingThread(threading.Thread):
    def process_frame(self, frame):
        # Step 1: Detect faces using Haar Cascade
        face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        face_locations = [(y, x+w, y+h, x) for (x, y, w, h) in faces]

        # Step 2: Encode faces with FaceNet
        encodings = self.encode_faces(frame, face_locations)

        # Step 3: Match against known faces
        results = []
        for location, encoding in zip(face_locations, encodings):
            name, confidence = self.match_face(encoding)
            results.append({
                'location': location,
                'name': name,
                'confidence': confidence
            })

        return results

    def match_face(self, face_encoding):
        '''Cosine similarity matching'''
        best_match = None
        best_distance = float('inf')

        with torch.no_grad():
            face_encoding = face_encoding.unsqueeze(0)

            for i, known_encoding in enumerate(self.known_encodings):
                # Cosine similarity (PyTorch optimized)
                similarity = torch.nn.functional.cosine_similarity(
                    face_encoding,
                    known_encoding.unsqueeze(0)
                )
                distance = 1.0 - similarity.item()

                if distance < best_distance:
                    best_distance = distance
                    best_match = i

        # Threshold: 0.40 (distance) / 0.75 (confidence minimum)
        if best_match is not None and best_distance < 0.40:
            name = self.known_names[best_match]
            confidence = 1.0 - best_distance
            if confidence >= 0.75:
                return name, float(confidence)  # Trusted

        return "Unknown", float(1.0 - best_distance)

### Screenshot: Dual Face Recognition

![Camera View](screenshots/camera.png)

**Key observations:**
- **Left (Green box):** aatmaj - confidence 0.89 (Trusted)
- **Right (Red box):** Unknown - confidence 0.24
- FPS: 18 (real-time performance on RTX 4050)
- Timestamp displayed: 2025-11-13 22:22:15
- Simultaneous multi-face detection working correctly

### 4. LLM Escalation Protocol (M3) - Jarvis AI

When an unknown face is detected for >2 seconds, the system enters a three-level escalation protocol powered by **Llama3.1:8b** via Ollama.

**Jarvis Personality System:**
1. **Stern Guard** - Direct, no-nonsense
2. **Professional Security** - Formal, protocol-focused
3. **Friendly Guard** - Polite but firm

**Escalation Levels:**
- **Level 1 (2s):** Initial contact + information gathering
- **Level 2 (4s):** Context-aware warning + exit demand
- **Level 3 (6s):** Final warning + authority notification

**Key Challenge:** Preventing LLM from **hallucinating names** when the intruder hasn't provided one.

In [ ]:
# Key Excerpt: Jarvis LLM Prompt Engineering (Anti-Hallucination)

def generate_llm_response(self, level, previous_speech, context):
    '''Generate Jarvis responses with strict constraints'''

    # Jarvis identity
    jarvis_intro = "You are Jarvis, an AI security assistant."

    # Anti-hallucination name handling
    if context.person_name:
        name_instruction = f"Use their name: {context.person_name}"
        name_example = f"Good: '{context.person_name}, you need to leave.'"
    else:
        # CRITICAL: Explicit instruction to prevent name hallucination
        name_instruction = "CRITICAL: No name given. Say 'you' or 'sir/ma'am' ONLY. Do NOT invent names."
        name_example = "Good: 'You need to leave now.' | Bad: 'John, leave now.' (hallucinated name)"

    if level == 1:
        prompt = f"""{jarvis_intro}
You have a {self.personality['tone']} personality.

SITUATION:
An unknown person appeared. You must verify their identity.

WHAT TO SAY (naturally cover these points):
- Inform them: "{self.personality['recording_notice']}"
- Ask who they are
- Ask why they're here

{name_instruction}

CRITICAL CONSTRAINTS:
- Maximum 20 words total (count them!)
- First person only (I, me, my)
- Start speaking immediately (no "Here's my response...")
- Be {self.personality['tone']}

Your response (20 words max):"""

    # Generate with Ollama
    response = ollama.generate(model='llama3.1:8b', prompt=prompt, stream=False)
    text = response.get('response', '').strip()

    # Post-processing: Enforce 20-word limit
    words = text.split()
    if len(words) > 20:
        text = ' '.join(words[:20]) + '.'

    return text

### Screenshot: Level 1 - Initial Contact

![Level 1](screenshots/level_1.png)

**Jarvis response:**
> "This interaction is recorded for security. Who are you and what brings you to this location?"

**Intruder response:**
> "My name is Roy and I am here to get the laptop."

**Context extraction:**
- ✅ Name extracted: "Roy"
- ✅ Purpose extracted: "retrieving electronics"

**Key observations:**
- Jarvis used recording notice (Professional Security personality)
- Asked for name AND purpose
- Response: 14 words (within 20-word limit)
- Listening with 5s countdown + silence detection (Phase 1 & 2)
- No name hallucination occurred

### Screenshot: Level 2 - Context-Aware Warning

![Level 2](screenshots/level_2.png)

**Jarvis response:**
> "Roy, I've been informed. You're not authorized to be here. Leave the premises immediately."

**Intruder response:**
> "Okay, I will leave now."

**Key observations:**
- Jarvis correctly used the name "Roy" from Level 1 context
- Acknowledged the situation but explained denial
- Demanded exit explicitly
- Response: 13 words (within limit)
- Timer: 34.3s accumulated presence
- NO name hallucination - used actual extracted name

### Screenshot: Level 2 - Escalation Drop (Trusted Entry)

![Level 2 Drop](screenshots/level_2_drop.png)

**Scenario:** During Level 2, a trusted face (aatmaj) entered the frame.

**System behavior:**
✅ Trusted entered - Dropping escalation  
✅ State returned to M2  
✅ Timer reset  
✅ No Level 3 triggered

**Key observations:**
- Escalation protocol correctly aborts when trusted face appears
- Clean state transition back to M2
- Accumulated timer resets
- Demonstrates dynamic response to changing conditions

### Screenshot: Level 3 - Final Warning

![Level 3](screenshots/level_3.png)

**Jarvis response:**
> "FINAL WARNING. Roy, I'm notifying authorities NOW. Leave IMMEDIATELY, as ignoring previous warnings is unacceptable."

**Key observations:**
- Used "FINAL WARNING" as required
- Still correctly using name "Roy" from context
- Explicitly stated authorities notified
- Emphasized urgency and consequences
- Response: 16 words (within limit)
- 30s cooldown activated after completion
- Professional Security personality maintained throughout

### 5. Deactivation & Clean Exit

The system supports two exit methods:
1. **Graceful deactivation:** Voice command "deactivate" + trusted face present
2. **Emergency shutdown:** Ctrl+C signal handler

In [ ]:
# Key Excerpt: Clean Deactivation Logic

def handle_m2(self):
    '''M2: Face monitoring with deactivation detection'''
    with self.camera_thread.lock:
        results = self.camera_thread.faces_data.copy()

    has_trusted = any(r['name'] != "Unknown" for r in results)

    if has_trusted:
        # Check if deactivation command heard
        transcriptions = self.audio_thread.get_deactivation_commands()

        for trans in transcriptions:
            similarity, matched_phrase = calculate_similarity(trans, DEACTIVATION_PHRASES)

            if similarity >= 0.6:
                print(f"✅ Deactivation (trusted face present)")

                # Pause audio before TTS (prevent echo)
                self.audio_thread.stop_listening()
                time.sleep(0.3)

                self.speak("System deactivated")

                # Clean thread shutdown
                self.camera_thread.stop()
                self.face_thread.stop()
                self.camera_thread.join(timeout=2)
                self.face_thread.join(timeout=2)

                self.state = State.IDLE
                return

# Ctrl+C handler
def signal_handler(self, sig, frame):
    '''Graceful shutdown on Ctrl+C'''
    print("\n🛑 Ctrl+C detected - Shutting down...")
    self.cleanup()

def cleanup(self):
    '''Clean resource release'''
    if self.camera_thread:
        self.camera_thread.stop()
        cv2.destroyAllWindows()
    if self.face_thread:
        self.face_thread.stop()
    if self.audio_thread:
        self.audio_thread.stop()
    torch.cuda.empty_cache()  # Release GPU memory

### Screenshot: Clean Deactivation

![Clean Exit](screenshots/clean_exit.png)

**Deactivation sequence:**
1. Heard: "deactivate guard"
2. Similarity: 0.84 (matched: "deactivate")
3. Deactivation triggered (trusted face: aatmaj present)
4. TTS: "System deactivated"
5. Face processing thread stopped
6. Camera thread stopped
7. System returned to IDLE → Personality selection screen

**Key observations:**
- Deactivation requires BOTH voice command AND trusted face
- Clean thread shutdown without crashes
- Returns to personality selection (ready for next activation)
- No memory leaks or zombie threads

### Screenshot: Ctrl+C Emergency Shutdown

![Ctrl+C](screenshots/ctrl_c.png)

**Emergency shutdown sequence:**
1. Ctrl+C signal caught
2. "Shutting down..." message
3. Audio thread stopped
4. Cleanup complete
5. Process terminated gracefully

**Key observations:**
- Signal handler prevents abrupt crashes
- Resources released properly (camera, threads, GPU memory)
- OpenCV windows closed
- No zombie processes or CUDA memory leaks

## Performance Metrics

### Frame Rate
- **Target:** 20 FPS
- **Achieved:** 18-20 FPS (average ~18 FPS)
- **Bottleneck:** Face detection (Haar Cascade on CPU)
- **Optimization:** Process every 2nd frame (reduces latency)

### GPU Memory Utilization
| Model | VRAM Usage | Percentage |
|-------|-----------|------------|
| Ollama (Llama3.1:8b) | 4.7 GB | 73.4% |
| Whisper Medium | 1.5 GB | 23.4% |
| FaceNet InceptionResnetV1 | 0.2 GB | 3.1% |
| **Total** | **6.4 GB** | **100%** |

### Latency
- **Voice activation detection:** <0.5s after command
- **Face recognition:** ~50ms per frame (20 FPS)
- **Whisper transcription:** ~0.5s per 3s audio chunk (GPU)
- **LLM response generation:** ~2-4s (Ollama on GPU)

### Accuracy
- **Face recognition:** 89% confidence on known faces (threshold: 75%)
- **Voice command similarity:** 0.84-0.89 on valid commands (threshold: 0.60)
- **Unknown detection:** 24% confidence correctly classified as "Unknown"


---

# Technical Report

## GPU Optimization & Integration Challenges

### 1. CUDA Out-of-Memory Error

**Challenge:**  
Initial attempts to load all three models resulted in:
```
RuntimeError: CUDA out of memory. Tried to allocate 4.70 GB (GPU 0; 6.00 GiB total capacity)
```

**Root Cause:**  
- PyTorch allocates GPU memory eagerly when `whisper.load_model(device="cuda")` is called
- If Ollama (4.7GB) loads after Whisper, CUDA runs out of contiguous memory
- Memory fragmentation due to random allocation order

**Solution:**  
Sequential loading with **Ollama → FaceNet → Whisper** order:
1. Ollama loads first (reserves 4.7GB before PyTorch initialization)
2. FaceNet loads with `torch.backends.cudnn.benchmark = True` (30% speedup)
3. Whisper loads last (fills remaining VRAM)

**Result:** 100% VRAM utilization without fragmentation or OOM errors.

---

### 2. Whisper Hallucination Problem

**Challenge:**  
Whisper Medium was generating false transcriptions:
- Input: Silence or "tomato ketchup"
- Output: "Thanks for watching!" or "I hope you enjoyed this video"

**Root Cause:**  
- Default Whisper parameters allow low-confidence outputs
- `condition_on_previous_text=True` causes context bias (repeats previous phrases)
- No `logprob_threshold` meant Whisper accepted <40% confidence guesses

**Solution:**  
Stricter transcription parameters:
```python
result = whisper_model.transcribe(
    audio,
    no_speech_threshold=0.8,          # Detect silence better
    logprob_threshold=-0.5,           # Require 60% confidence
    condition_on_previous_text=False, # No context bias
    temperature=0.0,                  # Deterministic (no sampling)
    compression_ratio_threshold=2.4   # Reject repetitive text
)
```

**Additional filter:** Minimum 2-word requirement (rejects single-word junk)

**Result:** Zero hallucinations in production testing.

---

### 3. TTS Echo Causing False Deactivation

**Challenge:**  
When Jarvis said "Guard mode activated," the microphone picked up the TTS audio, Whisper transcribed it as "a guard mode," and the system immediately deactivated itself.

**Root Cause:**  
Audio thread was still listening during TTS playback.

**Solution:**  
Pause/resume audio thread during TTS:
```python
def activate_guard(self):
    self.audio_thread.stop_listening()  # Pause
    time.sleep(0.3)  # Let in-flight chunks finish

    self.speak("Guard mode activated")  # TTS

    self.audio_thread.clear_deactivation_buffer()  # Clear old commands
    # ... start camera/face threads ...
    self.audio_thread.start_listening()  # Resume
```

**Result:** No false deactivations due to TTS echo.

### 4. LLM Name Hallucination

**Challenge:**  
When intruder didn't provide a name, Jarvis would say:
> "John, you need to leave immediately."

(Hallucinated "John" when no name was given)

**Root Cause:**  
LLM prompt said "Use their name if known" without explicit negative instruction.

**Solution:**  
Conditional name instruction with explicit anti-hallucination constraint:
```python
if context.person_name:
    name_instruction = f"Use their name: {context.person_name}"
else:
    name_instruction = "CRITICAL: No name given. Say 'you' or 'sir/ma'am' ONLY. Do NOT invent names."
    name_example = "Good: 'You need to leave now.' | Bad: 'John, leave now.' (hallucinated)"
```

**Additional safeguards:**
- Examples showing correct vs incorrect responses
- Word limit enforcement (20 words max)
- Post-processing to remove meta-commentary

**Result:** LLM correctly uses "you" when no name is provided, and actual names when extracted.

---

### 5. Thread Synchronization Issues

**Challenge:**  
Race conditions between camera thread (writing `faces_data`) and face thread (reading frames).

**Root Cause:**  
Unsynchronized shared memory access.

**Solution:**  
Thread-safe locks with `threading.Lock()`:
```python
class CameraThread:
    def __init__(self):
        self.lock = threading.Lock()
        self.faces_data = []

    def update_faces_data(self, faces):
        with self.lock:  # Atomic write
            self.faces_data = faces

    def get_faces_data(self):
        with self.lock:  # Atomic read
            return self.faces_data.copy()
```

**Result:** No data corruption or segmentation faults during concurrent access.

---

### 6. PyTorch/CUDA Version Compatibility

**Challenge:**  
Initial installation had:
- PyTorch 2.0.1 (CUDA 11.7)
- CUDA Toolkit 12.1 (system)
- Mismatch caused "CUDA driver version is insufficient" errors

**Root Cause:**  
PyTorch was compiled for CUDA 11.7 but system had CUDA 12.1 installed.

**Solution:**  
Reinstall PyTorch with correct CUDA version:
```bash
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
```

**Verification:**
```python
import torch
print(torch.cuda.is_available())  # True
print(torch.version.cuda)         # 12.1
```

**Result:** GPU fully accessible to all PyTorch models (FaceNet, Whisper).

## Model Selection Rationale

### Why Whisper Medium (not Base or Small)?

| Model | Size | VRAM | WER | Latency (3s audio) |
|-------|------|------|-----|-------------------|
| Whisper Base | 74M | 0.4GB | 16.9% | ~0.2s |
| Whisper Small | 244M | 0.8GB | 10.4% | ~0.3s |
| **Whisper Medium** | **769M** | **1.5GB** | **7.4%** | **~0.5s** |

**Decision:** Medium provides best accuracy/latency tradeoff for RTX 4050 (6.4GB VRAM).

### Why FaceNet (not MTCNN or DeepFace)?

| Model | Speed | Accuracy | VRAM | GPU Acceleration |
|-------|-------|----------|------|------------------|
| MTCNN | Slow | 95% | 0.3GB | Limited |
| DeepFace | Very Slow | 97% | 1.2GB | Limited |
| **FaceNet** | **Fast** | **93%** | **0.2GB** | **Full** |

**Decision:** FaceNet achieves 18-20 FPS on RTX 4050 with acceptable accuracy for security use case.

### Why Llama3.1:8b (not 3b or 70b)?

| Model | Size | VRAM | Response Quality | Latency |
|-------|------|------|-----------------|---------|
| Llama3.1:3b | 3B | 2.5GB | Fair | ~1s |
| **Llama3.1:8b** | **8B** | **4.7GB** | **Excellent** | **~3s** |
| Llama3.1:70b | 70B | 42GB | Best | ~20s |

**Decision:** 8b model fits in available VRAM (6.4GB after Whisper/FaceNet) and provides human-like responses.

## Future Improvements

### 1. Gait Recognition
Add **gait analysis** for continuous identity verification:
- Track walking patterns using OpenPose or MediaPipe
- Create gait signatures for known individuals
- Detect if "trusted" face is a photo/mask vs real person
- Reduce reliance on static face snapshots

### 2. Multi-Camera Support
- Deploy on multiple entry points simultaneously
- Centralized state machine with distributed camera nodes
- Synchronized alerts across all cameras

### 3. Cloud Integration
- Send L3 escalation events to cloud (Firebase/AWS)
- Store incident logs and video clips
- Mobile app notifications for homeowner

### 4. Edge Deployment
- Optimize for Jetson Nano / Raspberry Pi 5
- Quantize models to INT8 for lower VRAM
- Use TensorRT for 2x inference speedup

### 5. Voice Biometrics
- Add speaker verification (identify trusted voices)
- Combine with face recognition for 2-factor authentication
- Reduce false activations from TV/radio

## Conclusion

This project successfully implements a multi-modal AI security system with three integrated state-of-the-art models. Key achievements:

✅ **100% GPU utilization** (6.4GB / 6.4GB VRAM optimal allocation)  
✅ **Real-time performance** (18-20 FPS face recognition)  
✅ **Zero hallucinations** (both Whisper and LLM)  
✅ **Context-aware conversations** (Jarvis personality with memory)  
✅ **Robust state management** (clean transitions, no crashes)  

The system demonstrates practical applications of:
- **Voice AI:** Whisper for speech recognition
- **Computer Vision:** FaceNet for face recognition
- **Natural Language:** Llama3.1 for conversational AI
- **GPU Optimization:** Efficient memory management and model loading

Challenges including CUDA OOM errors, hallucinations, and thread synchronization were systematically solved through careful engineering and parameter tuning.

---

**Total Development Time:** 40+ hours  
**Lines of Code:** ~1,200  
**Models Integrated:** 3 (Whisper, FaceNet, Llama3.1)  
**Hardware:** NVIDIA GeForce RTX 4050 Laptop GPU (6.4GB VRAM)  
**Frameworks:** PyTorch, OpenCV, Ollama, Whisper, FaceNet-PyTorch